## 1. Importing libraries

In [15]:
import matplotlib.pyplot as plt
import plotly.express as px
import json
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import dateutil.parser

## 2. Reading data  
#### Post comments data from my Instagram

In [20]:
with open("post_comments.json", encoding='utf-8') as myFile:
  commentsData = json.load(myFile)
#printing a sample instance
#print(commentsData["comments_media_comments"][0]['string_map_data']['Comment']['value'])

## 3. Extracting comments from data
#### We are extracting only textual information from our data and ignoring timestamp parameter

In [21]:
print("The number of comments from 2016 to 2022 on my Instagram is : ", len(commentsData["comments_media_comments"]))
listComments = []
# converting complex json data to a list only with post comments
for i in range(len(commentsData["comments_media_comments"])):
  #extracting only comments data (textual info) from other parameters
  temp = commentsData["comments_media_comments"][i]['string_map_data']['Comment']['value']
  #removing the emojis from our comments
  temp = temp.encode('ascii',errors='ignore').decode()
  listComments.append(temp)
  temp = ""


The number of comments from 2016 to 2022 on my Instagram is :  1975


In [ ]:
#generating wordcloud for Instagram Post comments
def generateWordcloud(listComments):
  unique_string=(" ").join(listComments)
  wordcloud = WordCloud(width = 1000, height = 500).generate(unique_string)
  plt.figure(figsize=(15,8))
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.savefig("your_file_name"+".png", bbox_inches='tight')
  plt.show()
  plt.close()
  
generateWordcloud(listComments)

In [47]:
#extracting the timestamp from the comments posted from the json data
listTimestamp = []
for i in range(len(commentsData["comments_media_comments"])):
  temp = commentsData["comments_media_comments"][i]['string_map_data']['Time']['value']
  listTimestamp.append(temp)
  temp = ""

#parsing the timestamp
formatted_time = []
for i in range(len(commentsData["comments_media_comments"])):
  formatted_time.append(dateutil.parser.parse(dfTimestamp[0].iloc[i]))

#converting a list to a dataframe
dfTimestamp = pd.DataFrame(formatted_time)


In [49]:
dfTimestamp[0]

0        Apr 19, 2017, 3:35 AM
1        Apr 19, 2017, 2:53 AM
2       Apr 17, 2017, 12:22 PM
3        Apr 14, 2017, 8:40 AM
4         Apr 9, 2017, 2:07 AM
                 ...          
1970    Dec 20, 2021, 11:49 AM
1971     Dec 12, 2021, 8:53 PM
1972     Dec 12, 2021, 8:51 AM
1973     Nov 26, 2021, 7:15 AM
1974    Nov 17, 2021, 11:49 PM
Name: 0, Length: 1975, dtype: object

In [14]:
year,month,day,time = ([] for i in range(4))
from datetime import datetime
for i in range(len(formatted_time)):
  year.append(formatted_time[i].strftime("%Y"))
  month.append(formatted_time[i].strftime("%m"))
  day.append(formatted_time[i].strftime("%d"))
  time.append(formatted_time[i].strftime("%H:%M:%S"))


In [70]:
dfTimestamp_formatted = pd.DataFrame(list(zip(year, month, day, time)),columns=['year', 'month','day','time'])
dfTimestamp_formatted['month'] = pd.to_datetime(dfTimestamp_formatted['month'], format='%m').dt.month_name().str.slice(stop=3)

In [71]:
dfTimestamp_formatted.columns = dfTimestamp_formatted.columns.str.replace(' ', '')

In [1]:
yearly_group = dfTimestamp_formatted['year'].value_counts()
fig = px.bar(yearly_group, color=yearly_group,color_continuous_scale='orrd')
fig.update_layout(
    title = "Most Active In Comments By Year",
    autosize=False,
    width=700,
    height=500,
    xaxis_title="Year",
    yaxis_title="Number of Comments",
    fig.update_traces(textposition='top center')
    )

fig.show()


SyntaxError: positional argument follows keyword argument (<ipython-input-1-9364ffd0737d>, line 10)

In [2]:

by_time = dfTimestamp_formatted['month'].value_counts().sort_values(ascending = False)
fig = px.bar(by_time, color= by_time, color_continuous_scale='orrd')
fig.update_layout(
    title = "Most Active In Comments By Month",
    autosize=False,
    width=700,
    height=500,
    xaxis_title="Month", 
    yaxis_title="Number of Comments"
    )
fig.show()

NameError: name 'time_df_formatted' is not defined

In [118]:
by_day = dfTimestamp_formatted['day'].value_counts().head(5).sort_values(ascending = True)
fig = px.bar(by_day, orientation='h', color= by_day, color_continuous_scale='orrd')
fig.update_layout(
    title = "Most Active In Comments By Date",
    autosize=False,
    width=700,
    height=500,
    yaxis_title="Date", 
    xaxis_title="Number of Comments"
    )
fig.show()

In [74]:
dfTimestamp_formatted

,year,month,day,time
0,2017,Apr,19,02:53:00
1,2017,Apr,17,12:22:00
2,2017,Apr,14,08:40:00
3,2017,Apr,09,02:07:00
4,2017,Apr,07,19:50:00
...,...,...,...,...
1969,2021,Dec,20,11:49:00
1970,2021,Dec,12,20:53:00
1971,2021,Dec,12,08:51:00
1972,2021,Nov,26,07:15:00


In [94]:
timeNew = dfTimestamp_formatted['time'].str.split(":")

In [106]:
timeNew1 = []
for i in range(len(timeNew)):
  timeNew1.append(timeNew[i][0])
timeByHours = pd.Series(timeNew1)
dfTimestamp_formatted['timeByHours'] = timeByHours


In [117]:
dfTimestamp_formatted['timeByHours'].value_counts()

10    218
09    217
08    210
07    202
05    191
06    137
04     97
21     74
03     70
00     70
01     60
22     55
19     55
11     50
23     49
20     49
02     46
17     42
12     26
18     18
16     14
13     11
15      9
14      4
Name: timeByHours, dtype: int64

In [112]:
by_hours = dfTimestamp_formatted['timeByHours'].value_counts().sort_values(ascending = True)
fig = px.bar(by_hours, orientation='h', color= by_hours, color_continuous_scale='orrd')
fig.update_layout(
    title = "Most Active In Comments By Hours",
    autosize=False,
    width=700,
    height=500,
    xaxis_title="Number of Comments", 
    yaxis_title="Time in Hours"
    )
fig.show()